In [118]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/working/cat-and-dog-classifier/data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        #pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/working/cat-and-dog-classifier/data/Cat/why-cats-are-best-pets-1559241235.jpg
/kaggle/working/cat-and-dog-classifier/data/Cat/images128.jpg
/kaggle/working/cat-and-dog-classifier/data/Cat/images309.jpg
/kaggle/working/cat-and-dog-classifier/data/Cat/images254.jpg
/kaggle/working/cat-and-dog-classifier/data/Cat/images123.jpg
/kaggle/working/cat-and-dog-classifier/data/Cat/images413.jpg
/kaggle/working/cat-and-dog-classifier/data/Cat/images207(1).jpg
/kaggle/working/cat-and-dog-classifier/data/Cat/images29.jpg
/kaggle/working/cat-and-dog-classifier/data/Cat/images152.jpg
/kaggle/working/cat-and-dog-classifier/data/Cat/images149.jpg
/kaggle/working/cat-and-dog-classifier/data/Cat/images312.jpg
/kaggle/working/cat-and-dog-classifier/data/Cat/image29.jpeg
/kaggle/working/cat-and-dog-classifier/data/Cat/image12.jpeg
/kaggle/working/cat-and-dog-classifier/data/Cat/images392.jpg
/kaggle/working/cat-and-dog-classifier/data/Cat/images491.jpg
/kaggle/working/cat-and-dog-classifier/data/Ca

In [117]:
from PIL import Image
import os

# Directory containing the images
data_dir = '/kaggle/input/cat-and-dog-classifier/data/'

# Directory to save converted images
output_dir = '/kaggle/working/cat-and-dog-classifier/data/'

# List of subdirectories
subdirs = ['Dog', 'Cat']

# Iterate through each subdirectory
for subdir in subdirs:
    subdir_path = os.path.join(data_dir, subdir)
    
    # Check if the subdirectory exists
    if os.path.isdir(subdir_path):
        print(f"Processing images in folder: {subdir}")
        
        # Create output directory for the current subdirectory
        output_subdir = os.path.join(output_dir, subdir)
        os.makedirs(output_subdir, exist_ok=True)
        
        # Iterate through each image in the subdirectory
        for filename in os.listdir(subdir_path):
            filepath = os.path.join(subdir_path, filename)
            
            # Check if the file is an image
            if filepath.endswith(".png") or filepath.endswith(".jpg") or filepath.endswith(".jpeg"):
                # Open the image
                im = Image.open(filepath)
                
                # Convert the image to RGB format if it's not already in RGB
                if im.mode != 'RGB':
                    rgb_im = im.convert('RGB')
                    
                    # Save the converted image to the output directory
                    output_filepath = os.path.join(output_subdir, filename)
                    rgb_im.save(output_filepath)
                    print(f"{filename} converted to RGB and saved as {output_filepath}")
                else:
                    print(f"{filename} is already in RGB format, skipping conversion")
                
                output_filepath = os.path.join(output_subdir, filename)
                rgb_im.save(output_filepath)
            else:
                print(f"{filename} is not an image file, skipping")
    else:
        print(f"{subdir} folder does not exist, skipping")


Processing images in folder: Dog
images133.jpg is already in RGB format, skipping conversion
images412.jpg is already in RGB format, skipping conversion
images141.jpg is already in RGB format, skipping conversion
images356.jpg is already in RGB format, skipping conversion
images77.jpg is already in RGB format, skipping conversion
images210.jpg is already in RGB format, skipping conversion
images525.jpg is already in RGB format, skipping conversion
images272.jpg is already in RGB format, skipping conversion
images68.jpg is already in RGB format, skipping conversion
images360.jpg is already in RGB format, skipping conversion
images69.jpg is already in RGB format, skipping conversion
images396.jpg is already in RGB format, skipping conversion
images239.jpg is already in RGB format, skipping conversion
images494.jpg is already in RGB format, skipping conversion
images56.jpg is already in RGB format, skipping conversion
images22.jpg is already in RGB format, skipping conversion
images282.jp

In [119]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
from torchvision import transforms

# Hyper parameters
num_epochs = 20
batchsize = 64
lr = 0.001

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

DATA_PATH = "/kaggle/working/cat-and-dog-classifier/data/"

train_data = torchvision.datasets.ImageFolder(root=DATA_PATH, transform=TRANSFORM_IMG)
test_data = torchvision.datasets.ImageFolder(root=DATA_PATH, transform=TRANSFORM_IMG)

# Splitting the dataset into train and test sets
train_size = int(0.8 * len(train_data))
test_size = len(train_data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(train_data, [train_size, test_size])

train_data_loader = data.DataLoader(train_dataset, batch_size=batchsize, shuffle=True, num_workers=4)
test_data_loader = data.DataLoader(test_dataset, batch_size=batchsize, shuffle=True, num_workers=4)

# Data Loader (Input Pipeline)
train_loader = train_data_loader
test_loader = test_data_loader


In [120]:
for X, y in test_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 256, 256])
Shape of y: torch.Size([64]) torch.int64


In [121]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*256*256, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 3)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=196608, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=3, bias=True)
  )
)


In [122]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [52]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [31]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [124]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data_loader, model, loss_fn, optimizer)
    test(test_data_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 11.087511  [   64/  921]
Test Error: 
 Accuracy: 96.1%, Avg loss: 3.976411 

Epoch 2
-------------------------------
loss: 0.000000  [   64/  921]
Test Error: 
 Accuracy: 66.7%, Avg loss: 38.555441 

Epoch 3
-------------------------------
loss: 28.487280  [   64/  921]
Test Error: 
 Accuracy: 96.1%, Avg loss: 1.941588 

Epoch 4
-------------------------------
loss: 0.245132  [   64/  921]
Test Error: 
 Accuracy: 96.1%, Avg loss: 1.348548 

Epoch 5
-------------------------------
loss: 2.163684  [   64/  921]
Test Error: 
 Accuracy: 95.7%, Avg loss: 0.423905 

Done!


In [125]:
import torch
from torchvision import transforms
from PIL import Image

# Load the image
image_path = "/kaggle/input/cat-and-dog-classifier/data/dog_test2.jpg"
image = Image.open(image_path)

# Define the transformation

# Preprocess the image
input_image = TRANSFORM_IMG(image).unsqueeze(0)  # Add a batch dimension

# Move the input to the appropriate device (CPU or GPU)
input_image = input_image.to(device)

# Ensure the model is in evaluation mode
model.eval()

# Perform prediction
with torch.no_grad():
    output = model(input_image)

# Get the predicted class
_, predicted = torch.max(output, 1)

# Map the class index to the corresponding label
class_labels = ['Cat', 'Dog']
predicted_label = class_labels[predicted.item()]

print("Predicted Label:", predicted_label)


Predicted Label: Dog


In [126]:
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[ 0.0065,  0.0054,  0.0046,  ...,  0.0035,  0.0002,  0.0014],
                      [-0.0030, -0.0041, -0.0009,  ..., -0.0077, -0.0071, -0.0047],
                      [-0.0013,  0.0007, -0.0005,  ...,  0.0019,  0.0016,  0.0027],
                      ...,
                      [-0.0058, -0.0076, -0.0052,  ..., -0.0034, -0.0018, -0.0031],
                      [-0.0029, -0.0026, -0.0003,  ...,  0.0058,  0.0074,  0.0081],
                      [ 0.0055,  0.0019,  0.0042,  ...,  0.0020,  0.0030,  0.0008]])),
             ('linear_relu_stack.0.bias',
              tensor([ 1.2691e-03, -3.6298e-03, -1.1474e-02,  8.7088e-04, -2.7630e-03,
                      -1.0839e-02, -7.6152e-03, -4.6434e-03, -4.3392e-03, -9.6110e-03,
                      -6.9686e-03, -4.4532e-03, -6.8217e-03, -1.2309e-02, -3.0681e-03,
                      -1.1028e-02, -8.8761e-03, -1.2596e-02, -1.0291e-02, -1.0351e-02,
                      -4.9589e-0